<a href="https://colab.research.google.com/github/hwang-sudang/2020NH_Hackerton/blob/main/FinTalk/02)_%EA%B3%A0%EA%B0%9D%EA%B2%B0%EC%A0%9C%EB%8D%B0%EC%9D%B4%ED%84%B0_Fintalk_%ED%82%A4%EC%9B%8C%EB%93%9C_%EC%B6%94%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 드라이브 마운트, 데이터 갖고 오기

In [1]:
# data 불러오기
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd /content/drive/MyDrive/fintech_academy/NH/FinTalk/Data

/content/drive/.shortcut-targets-by-id/1V2gi8nQQ9c7k8o9PqnCU4MpVq7zscd-C/fintech_academy/NH/FinTalk/Data


In [3]:
import pandas as pd
import os
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1V2gi8nQQ9c7k8o9PqnCU4MpVq7zscd-C/fintech_academy/NH/FinTalk/Data'

In [4]:
!ls

 202008_usage.xlsx
'농협 주택화재보험_약관(20201101).doc'
'농협 주택화재보험_약관(20201101).pdf'
 202012_usage.xlsx
 card_all_data.xlsx
'현대해상 여행자보험 약관.doc'
'삼성화재 여행자보험.doc'
'삼성화재 주택화재보험.doc'
'한화생명 e연금저축보험(무)_1772-025_028_약관_20200401_           (2).doc'
 보험링크.gdoc
'IBK 연금저축형보험.txt'
 khaiii
'NH온라인연금저축보험(무배당)_2002_인쇄용 약관_v1_0204.doc'
'농협_무배당 NH지킴이재산종합보험2004_약관.doc'
'농협_무배당 NH지킴이재산종합보험2004_약관.pdf'
 on-off여행자보험.gdoc
 on-off여행자보험.txt
'현대해상 여행자보험 약관.pdf'
'삼성화재 여행자보험.pdf'
'삼성화재 주택화재보험.pdf'
 TF-IDF.docx
'감성분석 링크.txt'
'보험약관 TXT'
'이용대금명세서(신용카드).xls'
 여지헌카드내역.xlsx


# 01) 카드 소비 데이터 키워드 대분류, 소분류 분류작업
대분류, 소분류는 예시를 보여주기 위해 임의로 분류. 실무에서는 카드 결제 데이터 중 **업종코드**를 기준으로 키워드 대,소분류로 분류.
- 대분류 : 음식, 쇼핑, 자동차, 건강, 문화
- 소분류
  - 음식 : 편의점, 식당, 카페
  - 쇼핑 : 배달, 이커머스, 책
  - 자동차 : 대중교통, 자차
  - 건강 : 건강
  - 문화 : 문화


In [5]:
# 카드 결제 데이터 불러오기
card_all_data = pd.read_excel("card_all_data.xlsx")
card_all_data

,Unnamed: 0,이용일자,이용카드,가맹점,이용금액
0,0,20.10.27,마스터078,씨유 광명양지마을점,"3,000"
1,1,20.10.28,마스터078,NICE(빌링)-(주)모두의주차장,"5,000"
2,2,20.10.28,마스터078,택이네 조개전골 종로점,"45,000"
3,3,20.10.28,마스터078,교보문고,"22,000"
4,4,20.10.31,마스터078,안양원예농업협동조합 주유소,"10,000"
...,...,...,...,...,...
90,90,20.07.25,마스터078,제임스헤어,"13,000"
91,91,20.07.26,마스터078,요거프레소하안두산위브점,"7,000"
92,92,20.07.26,마스터078,요거프레소하안두산위브점,"1,000"
93,93,20.07.26,마스터078,재단법인 태성문화재단,100


In [6]:
# 필요한 정보인 가맹점, 이용금액만 추출
card_df = card_all_data[["가맹점", "이용금액"]]
card_df

,가맹점,이용금액
0,씨유 광명양지마을점,"3,000"
1,NICE(빌링)-(주)모두의주차장,"5,000"
2,택이네 조개전골 종로점,"45,000"
3,교보문고,"22,000"
4,안양원예농업협동조합 주유소,"10,000"
...,...,...
90,제임스헤어,"13,000"
91,요거프레소하안두산위브점,"7,000"
92,요거프레소하안두산위브점,"1,000"
93,재단법인 태성문화재단,100


In [7]:
# 가맹점 고유 갯수 확인
store_list = card_df["가맹점"].unique()
len(store_list)

68

In [8]:
# 가맹점열 raw data 내의 필요 없는 문자 제거
for i in card_df["가맹점"]:
  i = i.replace("\xa0","")

In [79]:
# 대분류 소분류 딕셔너리 : 데이터 구조상 사람이 임의로 분류 --> 실무에서는 업종코드로 대,소분류 가능.
dic = {"음식": {
"편의점":["씨유","지에스(GS)25"],
"식당":["조개전골","쉐이크쉑","한양중식","미미옥","60계치킨","스시장","여의도권초밥","바르다김선생","고반식당","진주식당","주식회사동빙고루시파이","교촌치킨", "갓잇", "빌즈","노메"],
"카페": ["파스쿠찌","투썸플레이스", "스타벅스", "커피온더스트릿", "파리바게뜨", "달콤커피", "요거프레소"]},
"자동차": {"대중교통":["지하철","버스","한국철도공사","철도"], "자차":["주차","주차장","아마노코리아","터널","파킹","주유소"],
"렌트카": ["그린카"]},
"쇼핑" : {
"배달":["(주)우아한형제들"],"이커머스":["쿠팡","11번가","롯데쇼핑","티몬","위메프","(주)에스에스지닷컴"],
"책":["교보문고", "오피스디포"],"간편결제":["모빌PG", "인터넷상거래", "KICC", "크린에이드", "카카오페이","스마트로", "이니시스"]},
"건강" : {"건강" : ["치과","약국"]},
"문화" : {"문화": ["재단법인","랭귀지랩","서울올림픽기념국민체육진흥공단"]}}

In [80]:
dic_df = pd.DataFrame([(k0, k1, v) for k0, d in dic.items() for k1, V in d.items() for v in V])
dic_df.head(15)

,0,1,2
0,음식,편의점,씨유
1,음식,편의점,지에스(GS)25
2,음식,식당,조개전골
3,음식,식당,쉐이크쉑
4,음식,식당,한양중식
5,음식,식당,미미옥
6,음식,식당,60계치킨
7,음식,식당,스시장
8,음식,식당,여의도권초밥
9,음식,식당,바르다김선생


In [24]:
# dic_df 기준으로 card_df 분류하기
class_list = []

#([(k0, k1, v) for k0, d in dic.items() for k1, V in d.items() for v in V])

for idx, dic in dic_df.iterrows():
  for idx2, store in card_df.iterrows():
    # 추후 계산을 위해 숫자형식 str > int 
    if type(store["이용금액"])==str : 
      tmp = (store["이용금액"]).replace(",","")
      store["이용금액"] = int(tmp)

    if dic[2] in store["가맹점"]:
      tmp = {"대분류": dic[0], "소분류": dic[1], "가맹점": store["가맹점"], "이용금액" : store["이용금액"] }
      class_list.append(tmp)
      

class_df = pd.DataFrame(class_list)
class_df


,대분류,소분류,가맹점,이용금액
0,음식,편의점,씨유 광명양지마을점,3000
1,음식,편의점,지에스(GS)25 신길역점,1800
2,음식,식당,택이네 조개전골 종로점,45000
3,음식,식당,쉐이크쉑 반포센트럴시티점,30000
4,음식,식당,한양중식,9500
...,...,...,...,...
97,건강,건강,광명파란문약국,3900
98,건강,건강,행복한수약국,4400
99,문화,문화,재단법인 태성문화재단,100
100,문화,문화,랭귀지랩(LanguageLab),3000


In [46]:
# 소분류 별로 이용금액 산출
grouped = class_df["이용금액"].groupby([class_df["대분류"],class_df["소분류"]]).sum()
grouped = pd.DataFrame(grouped)
# grouped.columns.drop_level([1,2])
grouped["소비점유율"]= round(grouped["이용금액"]/grouped["이용금액"].sum(),4)


# grouped["소비점유율"]=("%0.2f%%" % (grouped["이용금액"]/grouped["이용금액"].sum()))
grouped 

이용금액   소비점유율
대분류 소분류                 
건강  건강     17100  0.0145
문화  문화      7600  0.0064
쇼핑  간편결제   98000  0.0830
    배달     31848  0.0270
    이커머스  231273  0.1958
    책      51590  0.0437
음식  식당    449100  0.3801
    카페     86800  0.0735
    편의점     4800  0.0041
자동차 대중교통  104650  0.0886
    렌트카    18616  0.0158
    자차     80000  0.0677

# 01) 소분류를 기준으로 고객 분석 결과
## 주 소비군에서 키워드 선정
1. 식당 **>** #외식을 자주하는
2. 이커머스 **>** #쇼퍼홀릭
3. 대중교통 **>** #뚜벅이

## 실제 카드사에서 제공하는 데이터를 활용하면
- 업종코드별로 키워드를 객관적으로 지정하기 쉬울 것으로 기대된다. 
- 또한, 가맹점 정보가 더 많고 다양할수록 소분류 체계를 더욱 세분화 시킬 수 있을 것이다.
- 이러한 키워드 서칭 작업은 초기에는 마케팅, 기획 부서에서 인간의 주관에 따라 분류하고, 충분히 키워드 분류체계 라벨링 데이터가 확보되면 기계학습 분류 알고리즘으로 자동화 가능할 것이다.


# 02) 가맹점 데이터 형태소 분석을 통한 키워드 추출

### 형태소 분석기 사용을 위한 세팅

In [42]:
# 한글 표현을 위해 폰트 설치
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm


# 그래프에서 한글표현을 위해 폰트를 설치
%config inlineBackend.figure_format = "retina"
!apt-get -qq -y install fonts-nanum > /dev/null
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)

# 기본 글꼴 변경
mpl.pyplot.rc('font', family='NanumBarunGothic')
mpl.font_manager._rebuild()

1181377

In [ ]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

!pip install JPype1==0.7.4
!pip install rhinoMorph

## 형태소 분석 코드 시작
최빈 형태소를 기준으로 Fin-Talk 키워드를 뽑아낸다.

In [53]:
import rhinoMorph
rn = rhinoMorph.startRhino()

text="서울올림픽기념국민체육진흥공단"

morphs, poses = rhinoMorph.wholeResult_list(rn, text)


# text_analyzed = rhinoMorph.wholeResult_text(rn, text)
# print('\n.형태소 분석 결과:\n', text_analyzed)
print(morphs)

filepath:  /usr/local/lib/python3.6/dist-packages
classpath:  /usr/local/lib/python3.6/dist-packages/rhinoMorph/lib/rhino.jar
JVM is already started~
RHINO started!
['서울', '올림픽', '기념', '국민체육진흥공단']


In [67]:
# 가맹점 텍스트를 형태소 단위로 자르기
morphs_list = []

for idx, row in class_df.iterrows():
  text = row["가맹점"]
  text = text.replace("(","")
  text =text.replace(")","")
  text =text.replace("_","")
  text =text.replace("-","")
  text =text.replace("\xa0","")
  
  morphs, poses = rhinoMorph.wholeResult_list(rn, text)

  morphs_list.append(morphs)

class_df["형태소키워드"] = morphs_list

class_df

,대분류,소분류,가맹점,이용금액,형태소키워드
0,음식,편의점,씨유 광명양지마을점,3000,"[씨유, 광명, 을, 점]"
1,음식,편의점,지에스(GS)25 신길역점,1800,"[지, 에스, GS25, 신길역, 점]"
2,음식,식당,택이네 조개전골 종로점,45000,"[택이, 네, 조개, 로, 점]"
3,음식,식당,쉐이크쉑 반포센트럴시티점,30000,"[쉐이크, 쉑, 반포, 점]"
4,음식,식당,한양중식,9500,[한양중]
...,...,...,...,...,...
97,건강,건강,광명파란문약국,3900,"[광명, 파란, 약국]"
98,건강,건강,행복한수약국,4400,"[행복, 한수, 약국]"
99,문화,문화,재단법인 태성문화재단,100,"[재단법인, 태성, 문화, 재단]"
100,문화,문화,랭귀지랩(LanguageLab),3000,[랭귀지랩LANGUAGELAB]


In [77]:
# 최빈 형태소 10위 추출
morphs_keyword_list = []

for i in class_df["형태소키워드"]:
  # i = i.replace("\xa0","")
  morphs_keyword_list.extend(i)
  morphs_keyword_df = pd.DataFrame(morphs_keyword_list)

pd.DataFrame(morphs_keyword_df.value_counts()).head(10)

,0
0,
철도,17
한국,16
공사,16
주,16
주차장,14
점,12
이,11
그린,8
카,8


# 02) 최빈 빈출 키워드 분석결과
## 형태소로 분석한 주 소비군에서 키워드 선정
- 최빈 빈출 키워드를 기반으로 분석했을 때, 이 고객은 교통부문에 가장 많은 돈을 소비하고 있다.

- "철도" **>** #뚜벅이
- "주차장", "그린", "카" **>** #렌트카, #운전자보험

## 실제 카드사에서 제공하는 데이터를 활용하면
- 더욱 정교한 형태소 분석기를 활용하면 더 정확한 키워드 분석 가능
- 가맹점 정보가 더 많고 다양할수록 **불용어 처리, TF-IDF**를 통해 유의한 형태소만 추출한 후 분석하면 더욱 설득력 있는 키워드 추출 가능
- 이러한 키워드 서칭 작업은 초기에는 마케팅, 기획 부서에서 인간의 주관에 따라 분류하고, 충분히 키워드 분류체계 라벨링 데이터가 확보되면 기계학습 분류 알고리즘으로 자동화 가능할 것이다.
